In [1]:
import pandas as pd
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

In [2]:

import NeuralNetwork.pretrained_models as pre_models
from NeuralNetwork.tools import predict, predict_mul
from NeuralNetwork.preprocessing import skewness_pure, min_max_normalization_pure


def predict_df(X_df):

    model_A2_016, input_features_A2_016 = pre_models.Input_A2_016('weights/')
    
    outputs = ['Input_A2_016']

    preprocessings = ['skewness']

    models = [model_A2_016]

    input_features = [input_features_A2_016]

    result = {}

    for i in range(len(outputs)):

            if preprocessings[i] == 'skewness':
                X = skewness_pure(X_df, input_features[i])
            elif preprocessings[i] == 'min_max_normalization':
                X = min_max_normalization_pure(X_df, input_features[i])

            result[outputs[i]] = predict_mul(models[i], X)
    
    result_df = pd.DataFrame(result, columns = outputs)
    return result_df


In [3]:
def nn_ave(x, y):

    rmse = []
    r2 = []
    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(
            x, y, test_size=0.2, random_state=i)

        nn_pred = predict_df(X_test)

        rmse.append(math.sqrt(mean_squared_error(y_test, nn_pred)))
        r2.append(r2_score(y_test, nn_pred))

    test_rmse = np.array(rmse).mean()
    print('test_rmse_ave:', test_rmse)
    print(rmse)
    print('\n')
    test_r2 = np.array(r2).mean()
    print('test_r2_ave:', test_r2)
    print(r2)

In [4]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [5]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [6]:
fea_number=6
data=pd.read_csv('./BayesianRidge_Pre_0/Input_A2_016.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
nn=data.drop(['Predict'], axis=1)
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_A2_019', 'Input_A2_018', 'Input_A4_016', 'Input_A6_017',
       'Input_A6_016', 'Input_A5_019'], dtype=object)

In [7]:
test=pd.read_csv('./final_testX.csv',index_col=0).drop(['Number'], axis=1)[title] ## 改路徑

In [8]:
nn_ave(nn, y)

test_rmse_ave: 0.009261307764754794
[0.007230104583530393, 0.010564723086470796, 0.008943499966079684, 0.01125304553872289, 0.008829298875514577, 0.007651165296229191, 0.009351130639664171, 0.012602767173653643, 0.006781858247522159, 0.009405484240160421]


test_r2_ave: 0.47544583572434995
[0.7314067808041056, 0.3996935459687928, 0.4919767096937092, 0.23781244958081538, 0.548587952794346, 0.6607710406671781, 0.41838615503359244, 0.06458635056525841, 0.7108852181948312, 0.4903521539408704]


## Linear Regression Baseline

In [5]:
lr_rmse_ave_fea(skew_data,6)

train_rmse: 0.008044798708574738
test_rmse: 0.00842174415434341
test_r2: 0.5779946965784081


## Lasso

In [6]:
laso_rmse_ave(skew_data[title],y,5e-05)

test_rmse_ave: 0.008409586376601795
[0.008075465298330036, 0.00791152396369169, 0.008917094790309695, 0.008271623607208066, 0.007606711338365201, 0.007526075286801399, 0.009575456599207865, 0.009101761342332816, 0.00821787984201634, 0.008892271697754828]


test_r2_ave: 0.5770345886918602
[0.6649257948260741, 0.6633511990408596, 0.4949721011252771, 0.5881839904552834, 0.6649461746399874, 0.6717725733826536, 0.39014656858667873, 0.5121090481994248, 0.5754855726766388, 0.5444528639857233]


## ElasticNet

In [7]:
ElasticNet_rmse_ave(skew_data[title],y,[0.02],0.01)

test_rmse_ave: 0.00839153662939823
[0.00812996697614208, 0.007878619251568257, 0.008766092393325976, 0.008306545835593546, 0.007634577632503669, 0.007580113089361061, 0.009502217942602595, 0.009062295119254738, 0.008163387894411352, 0.008891550159219011]


test_r2_ave: 0.5794087765334078
[0.660387670622838, 0.6661456786234328, 0.5119315989675998, 0.5846993319781477, 0.6624868173694487, 0.6670422554959089, 0.39943991856589856, 0.5163309709842439, 0.5810967335417028, 0.5445267891848572]


## SVR

In [8]:
model=SVR(kernel='linear', C=1.2, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,6,model)

train_rmse: 0.008447081811618556
test_rmse: 0.008784714647108189
test_r2: 0.5381124015587389


## XGB

In [9]:
best_xgb_model = XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'gbtree', 
              n_estimators = 100, 
              max_depth = 5, 
              min_child_weight = 3,
              seed = 42,
              gamma = 0,
              subsample = 0.7,
              colsample_bytree = 0.65,
              reg_alpha =  0.02,
              reg_lambda = 1.5)
xgb_ave(skew_x, y,best_xgb_model)

test_rmse_ave: 0.00880764711951124
[0.008162968029984506, 0.008577762353002276, 0.009462903638912055, 0.00855081208142069, 0.007939599625002633, 0.008189740832580842, 0.009093054026034867, 0.009414726896857857, 0.009560799731481136, 0.009124103979835523]


test_r2_ave: 0.5373193386579707
[0.6576249751286939, 0.6042646958273833, 0.4312552000067853, 0.5599151457576778, 0.6349789465584614, 0.6113326421592595, 0.4500464240450244, 0.47797977488315724, 0.42540568727536754, 0.5203898949378959]


In [10]:
best_xgb_model = XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'gbtree', 
              n_estimators = 100, 
              max_depth = 5, 
              min_child_weight = 3,
              seed = 42,
              gamma = 0,
              subsample = 0.7,
              colsample_bytree = 0.65,
              reg_alpha =  0.02,
              reg_lambda = 1.5)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.0084989820590604
[0.00797442126676692, 0.007880498791456329, 0.008843353869953819, 0.008677508160940803, 0.007627026495941849, 0.007462491143935448, 0.009903671805447205, 0.009198473515063388, 0.008733635701649587, 0.008688739839448636]


test_r2_ave: 0.5664666091216988
[0.6732585480204027, 0.6659863696527217, 0.5032903342940361, 0.5467771895424935, 0.6631541359506046, 0.6772952125571414, 0.3476225101843011, 0.5016856415761222, 0.5205282479249806, 0.5650679015141846]


## RandomForest

In [26]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 7,
            min_samples_split = 5,
            max_leaf_nodes = 13,
            min_samples_leaf =2,
            random_state = 42) 


In [12]:
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.008385930877978835
[0.00815275039854773, 0.007539147462421775, 0.008919630790990507, 0.008243125803958515, 0.007255123396815386, 0.007471105994667943, 0.009491631810121028, 0.009025759235235686, 0.009240035168877976, 0.00852099871815182]


test_r2_ave: 0.5776245512724698
[0.6584815440648426, 0.6942959078459898, 0.49468480279621607, 0.5910167197248034, 0.6952032612383012, 0.6765497086239718, 0.4007773046823281, 0.520223065256559, 0.46331416968691996, 0.581699028804767]


In [9]:
lasso_mod=Lasso(alpha=5e-05)
elastic_mod=ElasticNet(alpha=[0.02], l1_ratio=0.01)
svr_mod=SVR(kernel='linear', C=1.2, gamma= 1e-07, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'gbtree', 
              n_estimators = 100, 
              max_depth = 5, 
              min_child_weight = 3,
              seed = 42,
              gamma = 0,
              subsample = 0.7,
              colsample_bytree = 0.65,
              reg_alpha =  0.02,
              reg_lambda = 1.5)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 7,
            min_samples_split = 5,
            max_leaf_nodes = 13,
            min_samples_leaf =2,
            random_state = 42) 
vote_mod = VotingRegressor([ ('Lasso', lasso_mod),('SVR', svr_mod), ('Elastic', elastic_mod), 
                            ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])

voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.00833724883538269
[0.008084329625178358, 0.0077167833044595666, 0.008938280403701121, 0.008189237338746791, 0.00731002841159538, 0.0073693064780852545, 0.009566930284664772, 0.00910991144159011, 0.008433505323239747, 0.008654175742565811]


test_r2_ave: 0.5832607358624321
[0.6641897784255105, 0.679720319828083, 0.4925695168697073, 0.5963466010187921, 0.6905725496032145, 0.6853041701492131, 0.39123215381942034, 0.5112349009143022, 0.552916000384611, 0.5685213676114664]


In [10]:

stack_mod = StackingRegressor(regressors=[lasso_mod, elastic_mod, random_mod, vote_mod,svr_mod], 
                           meta_regressor=xgb_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.00877764618323364
[0.008298944191014955, 0.007457161890719991, 0.00961201178762955, 0.009042321256450355, 0.007614750332940839, 0.007832295196865959, 0.010080647237287842, 0.009106523551399617, 0.009724607772991377, 0.009007198615035931]


test_r2_ave: 0.5350693843402643
[0.6461236218669866, 0.7009086044351112, 0.41319042308841325, 0.507868047523629, 0.6642376110972749, 0.6445194479429227, 0.32409863737289146, 0.5115983675828246, 0.4055476226990834, 0.5326014597935051]


In [11]:
ave(skew_data[title],y,0.2,stack_mod,0.5,vote_mod,0.3,random_mod)

test_rmse_ave: 0.008327598612661495
[0.0080465853101853, 0.007508462432894637, 0.008975380088964785, 0.008239541266570812, 0.007203989322316657, 0.0074083981227052575, 0.009547406042053572, 0.009017300087365546, 0.008776542941260707, 0.008552380512297687]


test_r2_ave: 0.5834513477905983
[0.667318136338634, 0.6967793316837203, 0.48834844090616825, 0.5913723365488566, 0.6994845325651113, 0.6819566078060154, 0.39371437173834123, 0.5211219596472856, 0.5158055079680286, 0.5786122527038212]


In [13]:
name='Input_A2_016'
test_pred=[]
seed=[0,1,4,5,3,8,9]
stack_w=0.2
vote_w=0.5
model_w=0.3
model=random_mod
for i in seed:
    X_train, X_test, y_train, y_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)

    stack = stack_mod.fit(X_train, y_train)
    stack_pred = stack.predict(test)
    vote = vote_mod.fit(X_train, y_train)
    vote_pred = vote.predict(test)
    mod = model.fit(X_train, y_train)
    mod_pred = mod.predict(test)

    ###
    final_test = (vote_w*vote_pred+stack_w*stack_pred + model_w*mod_pred)
    ###
    test_pred.append(final_test)



final_test = pd.DataFrame(np.mean(test_pred, axis=0), columns=[name])
final_test.to_csv(('./output/'+name+'.csv'))